In [145]:
import nltk
import gensim
import numpy as np
import pandas as pd

import pickle

from nltk.corpus import gutenberg, brown, reuters, stopwords

from collections import Counter, defaultdict
from pprint import pprint
from functools import reduce
from operator import itemgetter

from utils.nltk_utils import *
from utils.ngrams import *
from utils.eval import get_test_keywords
stop_words = stopwords.words('english')

In [146]:
import traceback
from typing import List, Tuple

from tqdm import tqdm

from utils.grammar_utils import tags_seq_to_symbols
from utils.io import Cache
from utils.ngrams import ngram2str


observed_tags = Cache.load_observed_tags()
terminal_rules = Cache.load_terminal_rules()


def parse_phrases(tt_ngrams) -> Tuple[List, List[List[Tuple]]]:
    global observed_tags, terminal_rules
    
    phrases = []
    phrases_types = []

    if observed_tags is None:
        observed_tags = dict()

    for tt_gram in tt_ngrams:
        symbols = tuple(tags_seq_to_symbols([tag
                                             for _, tag in tt_gram]))

        phrase = tt_gram

        # check if tags phrase has been already observed
        tags_str = ngram2str(symbols)

        if tags_str in observed_tags:
            if observed_tags[tags_str] is not None:
                phrases.append(phrase)
                phrases_types.append(observed_tags[tags_str])

            continue

        p_types_dict = terminal_rules.get(tags_str)

        if p_types_dict:
            p_type = max(p_types_dict, key=lambda k: p_types_dict[k])

            phrases.append(phrase)
            phrases_types.append(p_type)

            observed_tags[tags_str] = p_type

        else:
            observed_tags[tags_str] = None

#     try:
#         Cache.save_observed_tags(observed_tags)

#     except Exception as e:
#         traceback.print_exc(e)

    return phrases_types, phrases


# Preparing corpora

## Load data from NLTK 

In [4]:
sents = gutenberg.sents() + brown.sents() + reuters.sents()

In [5]:
sents = [tuple(s) for s in sents]

words = [w.lower() 
         for s in sents for w in s]

vocab = sorted(list(set(words)))

print('Sents:', len(sents))
print('Words:', len(words))
print('Vocab:', len(vocab))

Sents: 210608
Words: 5503894
Vocab: 87046


In [6]:
# creating indexes

# {sentence: sent_index}
sents_ind = {sents[i]: i for i in range(len(sents))}

# {word: word_index}
vocab_ind = {vocab[j]: j for j in range(len(vocab))}

## Create mapping from words to sentences

In [7]:
stemmer = Stemmer()

stems = [stemmer.stem(word) for word in vocab]

# {word_index: stem}
stems_map = {vocab_ind[word]: stem 
             for word, stem in zip(vocab, stems)}

print('Stems:', len(set(stems)))

Stems: 58858


In [8]:
stem2word = {stem: vocab[ind]
             for ind, stem in stems_map.items()}

## Word2Vec model

In [25]:
# loading
model = gensim.models.Word2Vec.load('data/w2v/CBOW_300_10_all')

In [14]:
sents_stemmed = [[stems_map[vocab_ind[w.lower()]] for w in s] for s in sents]

In [33]:
model = gensim.models.Word2Vec(sentences=sents_stemmed, size=300, window=15, min_count=1, hs=1, negative=0)
model.save('data/w2v/CBOW_300_10_hs_all')

In [34]:
# testing

print(model.wv.most_similar(positive=[stemmer.stem('woman'), stemmer.stem('king')], negative=[stemmer.stem('man')]))
print(model.wv.similarity(stemmer.stem('campus'), stemmer.stem('dormitory')))
print(model.wv.doesnt_match([stemmer.stem(w) for w in "dormitory bring campus".split()]))

[('queen', 0.3553354740142822), ('daughter', 0.35418325662612915), ('esther', 0.3361766040325165), ('husband', 0.3246108889579773), ('absalom', 0.3200189769268036), ('mordecai', 0.31594496965408325), ('samaria', 0.3088769316673279), ('vashti', 0.30666205286979675), ('hebron', 0.298782080411911), ('selleth', 0.29791125655174255)]
0.20989265750352995
bring


In [35]:
w2v = model.wv

In [19]:
del model

## Parsing corpora

In [ ]:
# load

with open('data/cache/stems_phrases', mode='rb') as fp:
    stems_phrases = pickle.load(fp)

In [9]:
parser = NGramsParser()

sents_words_indexes = parser.parse_sents_tokens(sents)

In [147]:
# {stem: Set[sentence_ind]}
stems_phrases = defaultdict(dict)

for i, (s_words, words_indexes) in tqdm(enumerate(zip(sents, sents_words_indexes), start=1)):
    words_ttokens = nltk.pos_tag([w.lower() for w in s_words])
    
    tt_ngrams = [ngr
                 for i in range(2, 5 + 1) 
                 for ngr in n_grams(words_ttokens, i, words_indexes, pad_left=False)]

    types, phrases = parse_phrases(tt_ngrams)

    # format and store phrases
    for t, p in zip(types, phrases):
        phrase_inds = tuple(vocab_ind[token] for token, _ in p)
        
        for word_ind in phrase_inds:
            if vocab[word_ind] not in stop_words:
                stem = stems_map[word_ind]
                
                if not stems_phrases[stem].get(t):
                    stems_phrases[stem][t] = defaultdict(set)
                
                stem_phr_t = stems_phrases[stem][t]
                
                stem_phr_t[phrase_inds].add(sents_ind[s_words])

210608it [06:48, 515.84it/s]


In [148]:
def get_phrases(word, stemmed=True):
    if stemmed:
        stem = word
    else:
        stem = stemmer.stem(word)
        
    phrs = stems_phrases[stem]
    
    if not phrs:
        return {}
    
    phrases_ = defaultdict(set)
    
    for p_type, phr_dict in phrs.items():
        for phrase, sents in phr_dict.items():
            phrase_ = tuple(vocab[ind] for ind in phrase)
            
            phrases_[p_type].add(phrase_)
            
    return phrases_

In [149]:
# save

with open('data/cache/stems_phrases', mode='wb') as fp:
    pickle.dump(stems_phrases, fp)

## Count number of phrases

In [150]:
# data for analysis

phrases_count = defaultdict(dict)

phrases_lengths_count = defaultdict(int)
phrases_types_count = defaultdict(int)

stems_phrases_count = defaultdict(dict)

for stem, stem_phrases_dict in stems_phrases.items():
    for phrase_type, phrases_dict in stem_phrases_dict.items():
#         stem_phrase_type_count = sum([len(s_ids) for s_ids in phrases_dict.values()])
        for phrase_tuple, s_ids in phrases_dict.items():
            phrase_len = len(phrase_tuple)
            phrase_sents_count = len(s_ids)
            
            if not stems_phrases_count[stem].get(phrase_len):
                stems_phrases_count[stem][phrase_len] = defaultdict(int)
                
            # count number of such phrase type for given stem
            stems_phrases_count[stem][phrase_len][phrase_type] += phrase_sents_count 

            # count total number of such phrase type
            phrases_count[phrase_len][phrase_type] = phrases_count[phrase_len].get(phrase_type, 0) + phrase_sents_count

            # count total number of all phrases
            phrases_lengths_count[phrase_len] += phrase_sents_count
            phrases_types_count[phrase_type] += phrase_sents_count

all_phrases_count = sum(phrases_types_count.values())

In [151]:
phrases_count

defaultdict(dict,
            {2: {'ADJP': 288142,
              'ADVP': 418156,
              'NP': 1985844,
              'PP': 3328,
              'VP': 66991},
             3: {'ADJP': 44770,
              'ADVP': 31064,
              'NP': 1353053,
              'PP': 7554,
              'VP': 8215},
             4: {'ADJP': 4179, 'ADVP': 223, 'NP': 429226},
             5: {'ADJP': 245, 'NP': 53025}})

In [152]:
WORD = 'share'

stems_phrases_count[stemmer.stem(WORD)]

{2: defaultdict(int,
             {'ADJP': 170, 'ADVP': 1833, 'NP': 5259, 'PP': 32, 'VP': 12}),
 3: defaultdict(int, {'ADJP': 13, 'ADVP': 4, 'NP': 3858, 'PP': 30}),
 4: defaultdict(int, {'NP': 879}),
 5: defaultdict(int, {'NP': 50})}

In [198]:
def get_word_phrases(word, stemmed=True):
    """
        Return sorted log probs of phrase types and length for the word 
    """
    if not stemmed:
        stem = stemmer.stem(word)
    else:
        stem = word
    
    if not stems_phrases_count.get(stem):
        return []
    
    stem_phrases = stems_phrases_count[stem]
    
    phrases_probs = []
    
    for p_length, p_types_count in stem_phrases.items():
        for p_type, p_type_count in p_types_count.items():
            prob_len = phrases_lengths_count[p_length] / all_phrases_count
            
            prob_len_type = phrases_count[p_length][p_type] / phrases_lengths_count[p_length]
            
            prob_word_len_type = p_type_count / phrases_count[p_length][p_type]
            
            log_prob = sum(np.log(p) for p in [prob_word_len_type, prob_len_type, prob_len])
            
            phrases_probs.append(((p_type, p_length), log_prob))
    
    phrases, scores = [], []
    for phr, s in sorted(phrases_probs, key=itemgetter(1), reverse=True):
        phrases.append(phr)
        scores.append(s)
    
#     scores = np.exp(scores)
#     scores /= np.max(scores)
    
    return list(zip(phrases, scores))

get_word_phrases(WORD, stemmed=False)

[(('NP', 2), -6.7941026773674675),
 (('NP', 3), -7.1038946574911375),
 (('ADVP', 2), -7.848089603117106),
 (('NP', 4), -8.583013953271635),
 (('ADJP', 2), -10.226000413906549),
 (('NP', 5), -11.449775845528665),
 (('PP', 2), -11.896062948157084),
 (('PP', 3), -11.960601469294655),
 (('ADJP', 3), -12.796849493495273),
 (('VP', 2), -12.87689220116881),
 (('ADVP', 3), -13.975504489836922)]

## Scoring phrases

In [204]:
LENGTH_PEN = {
    2: 1.75,
    3: 1.25,
    4: 0.75,
    5: 0.25
}
P_TYPE_SCORE = dict(get_word_phrases(WORD, stemmed=False))
REVERSE = False
SCORE_SIGN = -1
TOP_N = 5

phrases_ = get_phrases(WORD, stemmed=False)

all_scored_phrases = defaultdict(dict)

for p_type, phrases in phrases_.items():
    phrases_ = list(phrases)
    
    scores = np.array(model.score([[stemmer.stem(w) for w in phr] 
                                   for phr in phrases_]))

    for phr, sc in zip(phrases_, scores):
        if not all_scored_phrases[p_type].get(len(phr)):
            all_scored_phrases[p_type][len(phr)] = []
        
        score = SCORE_SIGN * (sc + P_TYPE_SCORE[(p_type, len(phr))]) * LENGTH_PEN[len(phr)] 
        
        all_scored_phrases[p_type][len(phr)].append((phr, score))

In [205]:
for p_type, scored_phrases in all_scored_phrases.items():
    print('%s\n' % p_type)
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

        best_phr = [(' '.join(p), s) for p, s in best_phr]

        print(n)
        pprint(best_phr)
        print()

NP

2
[('per share', 15.479596212664338),
 ('outstanding share', 17.877954676905368),
 ('outstanding shares', 17.877954676905368),
 ('shares outstanding', 17.877954676905368),
 ('share outstanding', 17.877954676905368)]

3
[('per share figure', 14.207894735781464),
 ('common share outstanding', 15.331119232622894),
 ('outstanding common share', 15.331119828669342),
 ('outstanding common shares', 15.331119828669342),
 ('share tender offer', 16.137549810854704)]

4
[('a share tender offer', 13.806982354449577),
 ('its outstanding common shares', 13.993565396594352),
 ('the outstanding common shares', 14.166183070468254),
 ('class b common share', 14.340641812609977),
 ('class b common shares', 14.340641812609977)]

5
[('a share cash tender offer', 6.792060174227014),
 ('all the outstanding common shares', 6.843968428850427),
 ('share cash tender offer today', 7.618016757250086),
 ('all common and preference shares', 7.650655783891931),
 ('a share cash takeover offer', 7.808050669908777)]

In [206]:
# group different length together

candidate_phrases = defaultdict(list)

for p_type, scored_phrases in all_scored_phrases.items():
    for n in scored_phrases:
        nps = scored_phrases[n]
        best_phr = sorted(nps, key=itemgetter(1), reverse=REVERSE)[:TOP_N]

        best_phr = [(' '.join(p), s) for p, s in best_phr]
        
        candidate_phrases[p_type].extend(best_phr)
        
    candidate_phrases[p_type] = sorted(candidate_phrases[p_type], key=itemgetter(1), reverse=REVERSE)
    
candidate_phrases

defaultdict(list,
            {'ADJP': [('share offered', 26.56781954052146),
              ('share converted', 31.566116516610933),
              ('purolator share outstanding', 31.798835104097485),
              ('share held', 32.0553107905596),
              ('share earned', 32.205990140458894),
              ('share dividend payable', 32.68091589747822),
              ('shared in', 33.704702680131746),
              ('share repurchase last', 41.04823977290547),
              ('share plus accrued', 42.21758992015278),
              ('share and multiple', 42.9320946389333)],
             'ADVP': [('per share', 17.324073332726208),
              ('of share', 25.63853557723686),
              ('sharing of', 25.638537246166912),
              ('share of', 25.638537246166912),
              ('for share', 26.89914436954185),
              ('nil per share', 34.81757354251771),
              ('ago per share', 38.967383618277594)],
             'NP': [('a share cash tender offer', 6.79206017

## Working with keywords

In [20]:
kws = get_test_keywords('data/lingualeo_words.csv')

print('Keywords: ', len(kws))

Keywords:  874


In [21]:
TEST_SIZE = 5
np.random.seed = 0

all_kws_found = False

while not all_kws_found:    
    test_kws = list(np.random.choice(list(kws), size=TEST_SIZE, replace=False))

    print('Test keywords:', test_kws)

    test_kws = [stemmer.stem(kw) for kw in test_kws]
    kws_phrases = [stems_phrases.get(kw) for kw in test_kws]
    print('Test keywords stems:', test_kws)

    all_kws_found = all(kws_phrases)
    
    print('All kws found:', all_kws_found)
    print()

Test keywords: ['advice', 'drab', 'shame', 'proclivity', 'admissible']
Test keywords stems: ['advic', 'drab', 'shame', 'procliv', 'admiss']
All kws found: True



In [22]:
def get_pairs_counter(test_kws):
    pairs = []

    for i, kw in enumerate(test_kws):
        cp = list(test_kws)
        cp.remove(kw)

        dsts = w2v.distances(kw, cp)
        print(kw, dsts)
        max_ind = np.argmin(dsts)
        print(i, max_ind)
        sim_kw = test_kws[max_ind + (1 if i <= max_ind else 0)]
        print(kw, sim_kw)

        pair = (kw, sim_kw)

        if (sim_kw, kw) in pairs:
            pair = (sim_kw, kw)

        pairs.append(pair)

    return Counter(pairs)

get_pairs_counter(test_kws)

advic [1.0484983  0.5876609  0.67798024 0.6448846 ]
0 1
advic shame
drab [1.0484983  0.85772705 0.5455078  0.85652786]
1 2
drab procliv
shame [0.5876609  0.85772705 0.9682493  0.80564225]
2 0
shame advic
procliv [0.67798024 0.5455078  0.9682493  0.66142946]
3 1
procliv drab
admiss [0.6448846  0.85652786 0.8056423  0.66142946]
4 0
admiss advic


Counter({('admiss', 'advic'): 1,
         ('advic', 'shame'): 2,
         ('drab', 'procliv'): 2})

## Using processed corpora

In [ ]:
# test

t = 'grievance'

s = stemmer.stem(t)
# stems_phrases[s]
vocab_ind['grievance']

In [ ]:
# test

t = 'grievance'

s = stemmer.stem(t)

ss = []
for k, v in stems_phrases[s]['NP'].items():
    ss.extend([s_id for s_id in v])
    
Counter(ss)